In [2]:
import random
import datetime
from dateutil.relativedelta import relativedelta
from faker import Faker
random.seed(49)
fake = Faker(seed=49,locale="es")

In [ ]:
formato_fecha = "%d-%m-%Y"

def str_a_date(fecha_str):
    d,m,y = fecha_str.split("-")
    return datetime.date(int(y), int(m), int(d))

def es_fecha_ultimos_anios(fecha, anios=2):
    hoy = datetime.datetime.today()
    fecha = str_a_date(fecha)
    anios_desde_fecha = relativedelta(hoy,fecha).years
    return anios_desde_fecha < anios

def get_fecha_ultimos_anios(anios=3, formato=formato_fecha): 
    return fake.date_between("-{}y".format(anios)).strftime(formato)

def get_precio(fecha, producto):
    """
        En base a una fecha y un codigo de producto, devuelve un precio simulando una inflacion random.
    """
    fecha_int = int((str_a_date(fecha)-datetime.date(2000,1,1)).total_seconds() / 1000000)
    return fecha_int * int(producto)

year = lambda x: x.split("-")[-1]

1- Se tiene un RDD con el registro de notas de los alumnos de la forma (padrón, materia, nota,fecha). Se pide resolver utilizando PySpark:

A. Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.

B. Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón,promedio).

C. El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un segundo RDD alumnos con registros (padron, nombre y apellido).

In [ ]:
cantidad_alumnos = 20
padrones = [ x for x in range(1, cantidad_alumnos+1) ]
materias = ("ALGEBRA I", "ANALISIS I", "ALGEBRA II", "ANALISIS II", "PROBABILIDAD", "ALGORITMOS I", "ALGORITMOS II")
notas = list(range(1,11))
fechas = ("01-02-2019", "01-07-2019", "01-02-2018", "01-07-2018", "01-12-2018", "01-02-2017", "01-07-2017", "01-12-2017")
def alumno_random(): return tuple([random.choice(x) for x in (padrones, materias, notas, fechas)])

cantidad_registros = 50
notas = { alumno_random() for a in range(cantidad_registros) }

rdd_notas = sc.parallelize(notas)

es_aprobado = lambda x: x > 3

ejercicio_a = rdd_notas\
    .filter(lambda x: es_aprobado(x[2]))\
    .filter(lambda x: es_fecha_ultimos_anios(x[3]))\
    .map(lambda x: (x[0], 0))\
    .groupByKey()\
    .count()

print("{} alumnos de {} aprobaron al menos 1 materia en los últimos 2 años".format(ejercicio_a, cantidad_alumnos))

ejercicio_b = rdd_notas\
    .map(lambda x: (x[0], (x[2], 1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+1))\
    .map(lambda x: (x[0], x[1][0]/x[1][1]))
# pide el RDD, no ejecuto ninguna acción
print(ejercicio_b)

nombres = [(x, fake.first_name(),fake.last_name()) for x in padrones]
rdd_nombres = sc.parallelize(nombres)

padron_mejor_promedio = ejercicio_b.max(lambda x: x[1])[0]
ejercicio_c = rdd_nombres.filter(lambda x: x[0]==padron_mejor_promedio).collect()[0]
print("'{}, {}' (padron {}) es el alumno con mejor promedio".format(ejercicio_c[2], ejercicio_c[1], ejercicio_c[0]))

---

2- Se tiene un RDD registros de ventas de producto con la forma (fecha de venta, código de  producto, precio de venta) y en otro RDD detalle de los productos con (código de producto, descripción del producto, categoría). Se pide resolver utilizando PySpark:

A. Cuál es el producto más vendido.

B. Cuál es la categoría de productos más vendida.

C. Cuál es el top5 de productos más vendidos generando un RDD con (código de producto, descripción, cantidad de ventas)

D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando para este análisis solo los productos que cuenten con al menos 50 ventas en el último año.

E. Idem anterior, pero calculando la categoría de productos que registró mayor variación de precios en el último año.

In [ ]:
cantidad_ventas = 20000
ids_productos = list(range(1, 11))
categorias = ["Autos", "Motos", "Lanchas", "Aviones"]
productos = [ (a,fake.street_name(),random.choice(categorias)) for a in ids_productos ]

def crear_venta():
    fecha, producto = get_fecha_ultimos_anios(), random.choice(ids_productos)
    return (fecha, producto, get_precio(fecha, producto))

ventas = [crear_venta() for x in range(cantidad_ventas)]
rdd_ventas = sc.parallelize(ventas)
rdd_productos = sc.parallelize(productos)

# A. Cuál es el producto más vendido.
codigo_mas_vendido = rdd_ventas.map(lambda x: (x[1], 0))\
    .reduceByKey(lambda x,y: x+1)\
    .max(lambda x: x[1])[0]
producto_mas_vendido = rdd_productos.filter(lambda x: x[0]==codigo_mas_vendido).collect()[0]
print("A) El producto más vendido es {}".format(producto_mas_vendido))

# B. Cuál es la categoría de productos más vendida.
categoria_mas_vendida = rdd_ventas.map(lambda x: (x[1], 0))\
    .join(rdd_productos.map(lambda x: (x[0], x[2])))\
    .map(lambda x: (x[1][1],0))\
    .reduceByKey(lambda x,y: x+1)\
    .max(lambda x: x[1])
print("B) La categoria mas vendida es {}, con {} ventas".format(*categoria_mas_vendida))

# C. Cuál es el top5 de productos más vendidos generando un RDD con (código de producto, descripción, cantidad de ventas)
top_5 = rdd_ventas.map(lambda x: (x[1], 0))\
    .reduceByKey(lambda x,y: x+1)\
    .top(5, lambda x: x[1])
top_5 = sc.parallelize(top_5).join(rdd_productos.map(lambda x: (x[0],x[1]))).map(lambda x: (x[0], *x[1])).collect()
print("C)", sorted(top_5, key=lambda x: x[1], reverse=True))

# D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando para este análisis solo los productos que cuenten con al menos 50 ventas en el último año.
# Voy a considerar promedio año actual vs promedio año anterior

ventas_ultimo_anio = rdd_ventas.filter(lambda x: es_fecha_ultimos_anios(x[0], 1)) #(fecha de venta, código de producto, precio de venta)

def min_max_ventas(x, y):
    """ no fue magia """
    fecha_min_x, venta_min_x, fecha_max_x, venta_max_x, cant_x = x
    fecha_min_y, venta_min_y, fecha_max_y, venta_max_y, cant_y = y

    min_x = str_a_date(fecha_min_x) < str_a_date(fecha_min_y)
    fecha_min = fecha_min_x if min_x else fecha_min_y
    venta_min = venta_min_x if min_x else venta_min_y
    
    max_x = str_a_date(fecha_max_x) > str_a_date(fecha_max_y)
    fecha_max = fecha_max_x if max_x else fecha_max_y
    venta_max = venta_max_x if max_x else venta_max_y
        
    return (fecha_min, venta_min, fecha_max, venta_max, cant_x+1)

codigo, aumento = ventas_ultimo_anio.map(lambda x: (x[1], (x[0],x[2],x[0],x[2],0)))\
    .reduceByKey(min_max_ventas)\
    .filter(lambda x: int(x[1][-1]) > 50)\
    .map(lambda x: (x[0],int(x[1][3])-int(x[1][1])))\
    .max(lambda x: x[1])
producto = rdd_productos.filter(lambda x: x[0]==codigo).collect()[0]
print("D) El producto {} registró un aumento de {} el último año".format(producto, aumento))

# E. Cuál es la categoria que registró mayor aumento de precio en el último año.
# Voy a considerar promedio año actual vs promedio año anterior

este_anio = datetime.date.today().year
anios = { str(este_anio-1), str(este_anio) }

actual = lambda x:  x==str(este_anio)

def mapper(x):
    """
        (cod, ((anio,precio),categoria)) -> (categoria, (suma_actual, suma_anterior, cant_actual, cant_anterior))
    """
    
    cod, ((anio, precio), categoria) = x
    es_actual = actual(anio)
    return (categoria, (precio if es_actual else 0, precio if not es_actual else 0, int(es_actual), int(not es_actual)))

def reducer(x,y):
    return tuple([sum(a) for a in zip(x,y)])

def map_dif(x):
    """
        (categoria, (suma_actual, suma_anterior, cant_actual, cant_anterior)) -> (categoria, aumento)
    """
    categoria, (suma_actual, suma_anterior, cant_actual, cant_anterior) = x
    avg_actual, avg_anterior = suma_actual/cant_actual , suma_anterior/cant_anterior
    aumento = (avg_actual/avg_anterior - 1) * 100
    return categoria, round(aumento,2)

aumentos = rdd_ventas.filter(lambda x: year(x[0]) in anios) \
    .map(lambda x: (x[1], (year(x[0]), x[2])))\
    .join(rdd_productos.map(lambda x: (x[0], x[2])))\
    .map(mapper)\
    .reduceByKey(reducer)\
    .map(map_dif)

max_aumento = aumentos.max(lambda x: x[1])

print("E) La categoria con más aumento ({}%) es '{}'".format(max_aumento[1], max_aumento[0]))

---
4- Se tiene un RDD con las coordenadas de rectángulos de la forma (x1,x2,y1,y2).

Se pide programar en PySpark un programa que encuentre el rectángulo de superficie mínima que contiene al punto (w,z)

In [ ]:
rectangulo_random = lambda: tuple((random.randint(-100,100) for a in range(4)))
cant_rectangulos = 200
rdd_rectangulos = sc.parallelize([rectangulo_random() for x in range(cant_rectangulos)])

superficie_rectangulo = lambda x: abs(x[3]-x[2])*abs(x[1]-x[0])
def contiene(rectangulo, punto) -> bool:
    """
        rectangulo: (x1,x2,y1,y2)
        punto: (x,y)
        ! todos enteros
    """
    x, y = punto
    min_x, max_x = sorted(rectangulo[:2])
    min_y, max_y = sorted(rectangulo[2:])
    return (min_x <= x <= max_x) and (min_y <= y <= max_y)

def encontrar_minimo(punto, rectangulos):
    """
        punto: (x,y)
        rectangulos: RDD [(x1,x2,y1,y2)]
    """
    minimo = rectangulos.filter(lambda x: contiene(x, punto))\
        .map(lambda x: (*x, superficie_rectangulo(x)))\
        .min(lambda x: x[-1])
    return minimo[:-1]

encontrar_minimo((3,6), rdd_rectangulos)

---
5- Se tiene un RDD con libros en donde cada registro es un texto. Se pide obtener todos los anagramas de mas de 7 letras que puedan encontrarse. El formato de salida debe ser una lista
de listas en donde cada lista tiene un conjunto de palabras que son anagramas. 

Ejemplo: [[discounter,introduces,reductions],[percussion,supersonic]...]

In [ ]:
"""

Quise hacer un fake para probar pero no da ni un anagrama de resultado:
# cant_textos, cant_caracteres = 100, 200000
# rdd_textos = sc.parallelize(fake.texts(cant_textos, cant_caracteres))

"""
rdd_textos = sc.parallelize(["TEXTOTEXTUAL OTROTEXTO pruebita", "TEXTOTEXTUAL XOTROTETO PUEBRITA"])
def text_to_clean_set(text, min_chars=7):
    cleaned = ""
    for char in text:
        cleaned += char.upper() if char.isalpha() else " "
    cleaned = cleaned.split()
    return {word for word in cleaned if len(word)>=min_chars}

anagramas = rdd_textos.flatMap(text_to_clean_set)\
    .map(lambda x: ("".join(sorted(x)), (x, set([x]))))\
    .reduceByKey(lambda x,y: (y[0], x[1].union(y[1])))\
    .map(lambda x: list(x[1][1]))\
    .filter(lambda x: len(x) > 1)

anagramas.collect()

---
6- UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus vehículos. Existe un proceso que nos devuelve un RDD llamado trip_summary con los siguientes campos: (driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled), Programar usando Py Spark un programa que nos indique cual fue el conductor con mayor promedio de distancia recorrida por viaje para Abril de 2016.

In [171]:
cant_viajes, cant_conductores, cant_clientes = 50000, 50, 2000
random_date = lambda: fake.date_between(start_date='-5y').strftime('%Y%m%d')
random_driver = lambda: random.choice(range(cant_conductores))
random_car = lambda: random.choice(range(cant_conductores))
random_customer = lambda: random.choice(range(cant_clientes))
random_distance = lambda: round(random.random()*20+1, 1)

trip_summary = sc.parallelize([(random_driver(), random_car(), viaje, random_customer(), random_date(), random_distance()) for viaje in range(cant_viajes)])

def top_monthly_driver(trip_summary, month):
    """
    Arguments:
        trip_summary: pyspark.rdd [driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled)]
        month: YYYYMM
    Returns:
        driver_id, avg_distance
    """
    driver_id, avg_distance = trip_summary.filter(lambda x: x[4][:-2]==month)\
        .map(lambda x: (x[0], (x[-1],1)))\
        .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
        .map(lambda x: (x[0], round(x[1][0]/x[1][1],2)))\
        .max(lambda x: x[1])
    return driver_id, avg_distance

driver_id, avg_distance = top_monthly_driver(trip_summary, "201604")
print("'{}' fue el conductor con mayor promedio de distancia por viaje ({}) para el mes solicitado".format(driver_id,avg_distance))

'31' fue el conductor con mayor promedio de distancia por viaje (15.59) para el mes solicitado


---
8- Contamos con un cluster que tiene 4 computadoras. Queremos aprovechar el paralelismo del cluster para calcular los números primos entre 2 y 20.000.000. Para esto usaremos el conocido algoritmo de la criba de Eratóstenes. Por ejemplo si empezamos con una lista de tipo (2,3,4,5,6,7,8...) en un primer paso eliminamos todos los que son mayores a 2 y divisibles por 2 y nos queda (2,3,5,7,9,11,13...) luego eliminamos todos los mayores a 3 divisibles por 3 y nos queda (2,3,5,7,11,13....etc) luego todos los divisibles por 5 y así sucesivamente. El resultado final es una lista de números que son primos. Programar este programa usando PySpark.

In [ ]:
# no hay caso

---
9- Se cuenta con un RDD con información sobre patentamientos de autos con la siguiente información (patente, marca, modelo, versión, tipo_vehiculo, provincia, fecha), donde tipo_vehiculo indica si la unidad patentada es auto, pickup, camión o moto. Se pide generar un programa en pySpark que indique la marca y modelo del auto más patentado por tipo de vehículo en la provincia de Buenos Aires en el mes de Abril de 2017.

In [252]:
random_patente = lambda: fake.license_plate()
provincias = ["BUENOS AIRES", "MENDOZA", "MISIONES", "CORDOBA", "CHUBUT"]
random_provincia = lambda: random.choice(provincias)
random_date = lambda: fake.date_between(start_date='-5y').strftime('%Y%m%d')
vehiculos = {
    "FORD": {
        ("AUTO","FIESTA"): ["SEDAN", "CABRIOLET"],
        ("AUTO","SOLO"): ["SEDAN", "CABRIOLET"],
        ("AUTO","LEOPARDO"): ["SEDAN", "CABRIOLET"],
        ("PICKUP","ECOSPORT"): ["SEDAN", "CABRIOLET"],
    },
    "MERCEDES": {
        ("PICKUP","CLASE A"): ["2.0","3.0"],
        ("PICKUP","CLASE B"): ["3.0","4.0"],
        ("CAMION","CLASE C"): ["SEDAN","COUPE"],
        ("MOTO","CLASE D"): ["2.0","3.0"],
        ("AUTO","CLASE E"): ["3.0","4.0"],
        ("AUTO","CLASE F"): ["SEDAN","COUPE"],
    },
    "FIAT": {
        ("MOTO","SIENA"): ["MINI", "POWER"],
        ("PICKUP","PUNTO"): ["CABRIO", "CLASSIC"],
        ("MOTO","ASTUTA"): ["MINI", "POWER"],
        ("CAMION","BRAVO"): ["CABRIO", "CLASSIC"]
    }
}

def random_vehiculo(vehiculos):
    marca = random.choice(list(vehiculos.keys()))
    tipo, modelo = random.choice(list(vehiculos[marca].keys()))
    version = random.choice(vehiculos[marca][(tipo,modelo)])
    return (marca,modelo,version,tipo)

cant_patentamientos = 50000
rdd_patentamientos = sc.parallelize([(random_patente(),*random_vehiculo(vehiculos),random_provincia(),random_date()) for x in range(cant_patentamientos)])

def mas_patentados(patentamientos, provincia, mes):
    """
    Arguments:
        rdd_patentamientos: pyspark.RDD [(patente, marca, modelo, versión, tipo_vehiculo, provincia, fecha<AAAAMMDD>)]
        provincia: str
        mes: str<AAAAMM>
    Returns:
        [(tipo_vehiculo, (marca, modelo, patentamientos))]
    """
    solucion = rdd_patentamientos.filter(lambda x: x[-2]==provincia and x[-1][:-2]==mes)\
        .map(lambda x: ((x[4],x[1],x[2]), 1))\
        .reduceByKey(lambda x,y: (x+y))\
        .map(lambda x: (x[0][0], (x[0][1],x[0][2],x[1])))\
        .reduceByKey(lambda x,y: x if x[-1]>y[-1] else y)\
        .collect()
    return solucion

for tipo, (marca, modelo, patentamientos) in mas_patentados(rdd_patentamientos, "BUENOS AIRES", "201704"):
    print("{}: {} {} con {} patentamientos".format(tipo,marca,modelo,patentamientos))

MOTO: FIAT ASTUTA con 17 patentamientos
PICKUP: FORD ECOSPORT con 16 patentamientos
CAMION: MERCEDES CLASE C con 15 patentamientos
AUTO: FORD SOLO con 13 patentamientos
